https://archive.ics.uci.edu/ml/datasets/wine

In [1]:
# generation of rule items along with their support count.


class RuleItems:
    
    # every ruleitem have
    # 1. y -> class label
    # 2. condset -> a particular set of k items
    # will output
    # 1. condsupCount -> number of times condset appears
    # 2. rulesupCount -> number of times condset appears with a class label
    # 3. support -> % of data containing condset
    # 4. confidence -> % of data that have class label given condset
    

    def __init__(self, condset, class_label, dataset):
        self.condset = condset
        self.class_label = class_label
        self.condsupCount, self.rulesupCount = self.get_supCount(dataset)
        self.support = self.get_support(len(dataset))
        self.confidence = self.get_confidence()
        
    def get_supCount(self, dataset):
        rulesupCount =0;
        condsupCount = 0;
        
        for row in dataset:
            contain = True
            for i in self.condset:
                if self.condset[i] != row[i]: # want to compare same row
                    contain = False
                    break;
            if contain:
                condsupCount += 1;
                if self.class_label == row[-1]: # same as class label 
                    rulesupCount +=1;
        return condsupCount, rulesupCount
    
    def get_support(self, length):
        return self.rulesupCount/length;
    
    def get_confidence(self):
        if self.condsupCount != 0:
            return self.rulesupCount / self.condsupCount;
        else:
            return 0;

    # print out rule
    def print_rule(self):
        cond_set_output = ''
        for item in self.condset:
            cond_set_output += '(' + str(item) + ': ' + str(self.condset[item]) + '), '
            
        cond_set_output = '{' + cond_set_output[:-2] + '}'
        print(cond_set_output + ' -> (class, ' + str(self.class_label) + ')')
        
    def printAll(self):
        cond_set_output = ''
        for item in self.condset:
            cond_set_output += '(' + str(item) + ', ' + str(self.condset[item]) + '), '
        cond_set_output = cond_set_output[:-2]
        print('<({' + cond_set_output + '}, ' + str(self.condsupCount) + '), (' +
              '(class, ' + str(self.class_label) + '), ' + str(self.rulesupCount) + ')>')
        # output: {(9: 90.0), (2: 45.0)} -> (class, 1150.0)
        
    

In [2]:
# storing frequent rule items using set

class FrequentRuleItems:

    def __init__(self):
        self.frequentRuleItems = set();

    def add(self, rule_item):
    # check for duplicates 
        dup = False;
        for item in self.frequentRuleItems:
            if item.class_label == rule_item.class_label:
                if item.condset == rule_item.condset:
                    dup = True;
                    break;
        # not a duplicate, add 
        if dup == False:
            # print("Not a duplicate")
            # print("Adding the following rule item to the frequent rule items...")
#             print(str(rule_item.condset) + ' -> class: ' + str(rule_item.class_label))
            self.frequentRuleItems.add(rule_item);

    def get_size(self):
        return len(self.frequentRuleItems);

    # print out all frequent ruleitems
    def print(self):
        for item in self.frequentRuleItems:
            item.print_rule()

    # append set of ruleitems
    def append(self, sets):
        for item in sets.frequent_ruleitems:
#             print("printing item")
#             print(item)
            self.add(item)


In [22]:

# prune rule
class PrunedRules:

    def __init__(self):
        self.rules = set() # final set of rules
        self.pruned_rules = set()

    def pruning(self, rule_item, minSup, minCon): # pruning: selecting unique condsets that meets the min supp and conf
        if rule_item.support >= minSup and rule_item.confidence >= minCon: # check if meet min support and min confidence
            if rule_item in self.rules: # dup item
                return; # dont add
            
            for item in self.rules:
                # compares condset, TODO:what about class? 
                if rule_item.condset == item.condset:          
                    if rule_item.confidence > item.confidence: # check confidence # select rule with highest confidence or first rule if same confidence
                        self.rules.remove(item);
                        self.rules.add(rule_item);
                        return;
                    else:
                        return;

            self.rules.add(rule_item); # set only adds unique items


    def generate_rules(self, rule_items, minSup, minCon): # entry point
        for freqitem in rule_items.frequentRuleItems:
              self.pruning(freqitem, minSup, minCon);

    # print out all pruned rules
    def print_pruned_rule(self):
        for item in self.pruned_rules:
            item.print_rule()

    # print out all rules
    def print_rule(self):
        for item in self.rules:
            item.print_rule()

    # union new car into rules list
    def append(self, prune, minsup, minconf):
        for item in prune.rules:
            self.pruning(item, minsup, minconf)

    # prune rules
    def prune_rules(self, dataset):
        for rule in self.rules:
            pruned_rule = prune(rule, dataset)

            is_existed = False
            for rule in self.pruned_rules:
                if rule.class_label == pruned_rule.class_label:
                    if rule.condset == pruned_rule.condset:
                        is_existed = True
                        break

            if not is_existed:
                self.pruned_rules.add(pruned_rule)
                
    def prune_rules_improve(self): # take maximum
        improve_prune = set()
        sorted_rules = self.pruned_rules.sort(key=lambda x: x.class_label, reverse=True)
        print("Is this running?")
        self.pruned_rules = sorted_rules
        
#         prCARs:
# {(0: 0.0), (9: 90.0), (2: 45.0), (5: 18.0)} -> (class, 1150.0)
# {(0: 0.0), (9: 90.0), (4: 2.0), (5: 18.0)} -> (class, 1150.0)
# {(0: 0.0), (2: 45.0), (4: 2.0)} -> (class, 1150.0)
# {(0: 0.0), (9: 90.0), (4: 2.0)} -> (class, 1150.0)
# {(0: 0.0), (9: 90.0), (2: 45.0), (4: 2.0)} -> (class, 1150.0)
# {(0: 0.0), (4: 2.0), (5: 18.0)} -> (class, 1150.0)
# {(0: 0.0), (4: 2.0)} -> (class, 1150.0)
# {(2: 45.0), (5: 18.0)} -> (class, 1150.0)
# {(0: 0.0), (5: 18.0)} -> (class, 1150.0)
# {(4: 2.0), (5: 18.0)} -> (class, 1150.0)
# {(0: 0.0), (2: 45.0), (4: 2.0), (5: 18.0)} -> (class, 1150.0)
# {(0: 0.0), (9: 90.0)} -> (class, 1150.0)
# {(9: 90.0), (4: 2.0)} -> (class, 1150.0)
# {(9: 90.0), (2: 45.0)} -> (class, 1150.0)
# {(9: 90.0), (5: 18.0)} -> (class, 1150.0)
# {(0: 0.0), (2: 45.0), (4: 2.0), (5: 18.0), (9: 90.0)} -> (class, 1150.0)
# {(4: 2.0)} -> (class, 1150.0)
# {(9: 90.0), (2: 45.0), (5: 18.0)} -> (class, 1150.0)
# {(2: 45.0), (4: 2.0), (5: 18.0)} -> (class, 1150.0)
# {(9: 90.0), (2: 45.0), (4: 2.0)} -> (class, 1150.0)
# {(9: 90.0), (4: 2.0), (5: 18.0)} -> (class, 1150.0)
# {(0: 0.0), (9: 90.0), (2: 45.0)} -> (class, 1150.0)
# {(2: 45.0), (4: 2.0)} -> (class, 1150.0)
# {(0: 0.0), (2: 45.0), (5: 18.0)} -> (class, 1150.0)
# {(9: 90.0), (2: 45.0), (4: 2.0), (5: 18.0)} -> (class, 1150.0)
# {(0: 0.0), (9: 90.0), (5: 18.0)} -> (class, 1150.0)

# {(0: 0.0), (7: 77.0)} -> (class, 1060.0)

# {(0: 4.0), (9: 21.0)} -> (class, 650.0)

# {(11: 24.0)} -> (class, 515.0)

# {(0: 4.0), (11: 18.0)} -> (class, 600.0)


# {(0: 2.0), (12: 70.0)} -> (class, 345.0)
# {(12: 70.0)} -> (class, 345.0)

# {(11: 65.0), (4: 12.0)} -> (class, 428.0)
# {(0: 2.0), (11: 65.0), (4: 12.0)} -> (class, 428.0)

# {(1: 38.0)} -> (class, 450.0)
# {(0: 2.0), (1: 38.0)} -> (class, 450.0)

# {(9: 25.0), (5: 13.0)} -> (class, 562.0)
# {(0: 2.0), (9: 25.0), (5: 13.0)} -> (class, 562.0)


# {(1: 3.0)} -> (class, 680.0)
# {(11: 38.0), (4: 40.0)} -> (class, 680.0)
# {(0: 2.0), (1: 3.0)} -> (class, 680.0)


# {(0: 4.0), (2: 92.0)} -> (class, 675.0)
# {(2: 92.0), (4: 29.0)} -> (class, 675.0)
# {(0: 4.0), (2: 92.0), (4: 29.0)} -> (class, 675.0)

# {(1: 88.0), (2: 22.0)} -> (class, 1285.0)
# {(1: 88.0)} -> (class, 1285.0)
# {(0: 0.0), (1: 88.0)} -> (class, 1285.0)
# {(0: 0.0), (1: 88.0), (2: 22.0)} -> (class, 1285.0)

In [4]:
# invoked by candidate_gen, join two items to generate candidate
def join(item1, item2, dataset):
    # check class
    if item1.class_label != item2.class_label:
        return None

    category1 = set(item1.condset)
    category2 = set(item2.condset)
    
    # check condset
    if category1 == category2:
        return None
    # example:
    # cat1 = {0: 1, 1: 2, 2: 1}
    # cat2 = {0: 1, 1: 1, 2: 1}
    # intersect = {0: 1, 2: 1}, the intersect of 2 sets
    intersect = category1 & category2
    # item refers to the column index or attribute
    for item in intersect:
        # example:
        # item = 1, item1.condset[1] = {1: 2}, item2.condset[1] = {1: 1}
        # ADB -> 1, ACB -> 1
        if item1.condset[item] != item2.condset[item]:
            return None
    
    # ABC -> 1, ABD -> 1
    # union: {0: A, 1: B, 2: C, 2: D}
    union_cat = category1 | category2
    new_cond_set = dict()

    for item in union_cat:
        if item in category1:
            new_cond_set[item] = item1.condset[item]
        else:
            new_cond_set[item] = item2.condset[item]
    new_ruleitem = RuleItems(new_cond_set, item1.class_label, dataset)
    return new_ruleitem


# similar to Apriori-gen in algorithm Apriori
def candidate_gen(frequent_ruleitems, dataset):
    returned_frequent_ruleitems = FrequentRuleItems()
    for item1 in frequent_ruleitems.frequentRuleItems:
        for item2 in frequent_ruleitems.frequentRuleItems:
            new_ruleitem = join(item1, item2, dataset)
            if new_ruleitem:
                returned_frequent_ruleitems.add(new_ruleitem)
                if returned_frequent_ruleitems.get_size() >= 1000:      # not allow to store more than 1000 ruleitems
                    return returned_frequent_ruleitems
    return returned_frequent_ruleitems

In [5]:
import sys

def is_satisfy(datacase, rule):
    for item in rule.condset:
        if datacase[item] != rule.condset[item]:
            return None
    if datacase[-1] == rule.class_label:
        return True
    else:
        return False

# calculate how many errors the rule r make in the dataset
def errors_of_rule(r):
    errors_number = 0
    for case in dataset:
        if is_satisfy(case, r) == False:
            errors_number += 1
    return errors_number

# prune rule recursively
def find_prune_rule(this_rule):
    min_rule_error = sys.maxsize
    rule_error = errors_of_rule(this_rule)
    if rule_error < min_rule_error:
        min_rule_error = rule_error
        pruned_rule = this_rule
    this_rule_cond_set = list(this_rule.condset)
    if len(this_rule_cond_set) >= 2:
        for attribute in this_rule_cond_set:
            temp_cond_set = dict(this_rule.condset)
            temp_cond_set.pop(attribute)
            temp_rule = RuleItems(temp_cond_set, this_rule.class_label, dataset)
            temp_rule_error = errors_of_rule(temp_rule)
            if temp_rule_error <= min_rule_error:
                min_rule_error = temp_rule_error
                pruned_rule = temp_rule
                if len(temp_cond_set) >= 2:
                    find_prune_rule(temp_rule)

# try to prune rule
def prune(rule, dataset):
    min_rule_error = sys.maxsize
    pruned_rule = rule
    find_prune_rule(rule)
    return pruned_rule

In [6]:
# main method
def cba_rg(dataset, minsup, minconf):
    frequent_ruleitems = FrequentRuleItems()
    car = PrunedRules()

    # get large 1-ruleitems and generate rules
    class_label = set([x[-1] for x in dataset])
    for column in range(0, len(dataset[0])-1):
        distinct_value = set([x[column] for x in dataset])
        for value in distinct_value:
            cond_set = {column: value}
            for classes in class_label:
                rule_item = RuleItems(cond_set, classes, dataset)            
                if rule_item.support >= minsup:
                    frequent_ruleitems.add(rule_item)
#     print("All 1-rule items:")
#     frequent_ruleitems.print()
    car.generate_rules(frequent_ruleitems, minsup, minconf)
#     print("Pruned 1-rule items:")
#     car.print_pruned_rule()
    # save an instance of the car to cars
    cars = car
#     print("Current number of rules: " + str(len(cars.rules)))
    
    last_cars_number = 0
    # pruned rules number
    current_cars_number = len(cars.rules)
    # for debugging
    k = 2
    
    # .get_size() returns the number of frequent rules left
    # (current_cars_number - last_cars_number) 
    while frequent_ruleitems.get_size() > 0 and current_cars_number <= 2000 and \
                    (current_cars_number - last_cars_number) >= 5:
        candidate = candidate_gen(frequent_ruleitems, dataset)
        frequent_ruleitems = FrequentRuleItems()
        car = PrunedRules()
        for item in candidate.frequentRuleItems:
            if item.support >= minsup:
                frequent_ruleitems.add(item)
        
#         print(f"All {k}-rule items:")
#         frequent_ruleitems.print()
        car.generate_rules(frequent_ruleitems, minsup, minconf)
#         print(f"Pruned {k}-rule items:")
#         car.print_pruned_rule()

        # Adding all the rules together
        cars.append(car, minsup, minconf)
        last_cars_number = current_cars_number
        current_cars_number = len(cars.rules)
        k += 1

    return cars


In [23]:
dataset = [[1, 1, 1,2], [1, 1, 2,2], [2, 1, 2,1], [1, 2, 2,1], [3, 1, 1,1],
               [1, 1, 1,2], [2, 2, 3,1], [1, 2, 3,1], [1, 2, 2,1], [1, 2, 2,2]]
minsup = 0.01
minconf = 0.6
cars = cba_rg(dataset, minsup, minconf)

# print("CARs:")
# cars.print_rule()

print("prCARs:")
cars.prune_rules(dataset)
cars.print_pruned_rule() # final rules generated

prCARs:


AttributeError: 'set' object has no attribute 'sort'